In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
kc_housing = pd.read_csv('kc_house_data.csv')
kc_housing.head().T

,0,1,2,3,4
id,7129300520,6414100192,5631500400,2487200875,1954400510
date,10/13/2014,12/9/2014,2/25/2015,12/9/2014,2/18/2015
price,221900,538000,180000,604000,510000
bedrooms,3,3,2,4,3
bathrooms,1,2.25,1,3,2
sqft_living,1180,2570,770,1960,1680
sqft_lot,5650,7242,10000,5000,8080
floors,1,2,1,1,1
waterfront,NaN,0,0,0,0
view,0,0,0,0,0


In [13]:
# Housing inflation correction?
# Look at potential correlation with all options
# To drop - Lat/long(or convert to neighborhood? - have ZIP code),
# ID, date (transform and just grab year)
# Potentially convert year built to age?
# Look for empty, bad, values
# Potentially log price, bedrooms? Basement
# FIgure out sqft_above
# Categorical - condition, grade (subjective?), bedrooms, bathrooms, floors,

In [7]:
# Drop some columns
kc_housing = kc_housing.drop(columns=['id', 'lat', 'long'])
kc_housing['date'] = kc_housing['date'].apply(lambda x: x[-4:])

In [19]:
kc_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 18 columns):
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(6), int64(10), object(2)
memory usage: 3.0+ MB


In [20]:
kc_housing.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,1340,5650
1,2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,1690,7639
2,2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,2720,8062
3,2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,1360,5000
4,2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,1800,7503


In [21]:
kc_housing.isna().sum(), kc_housing.shape

(date                0
 price               0
 bedrooms            0
 bathrooms           0
 sqft_living         0
 sqft_lot            0
 floors              0
 waterfront       2376
 view               63
 condition           0
 grade               0
 sqft_above          0
 sqft_basement       0
 yr_built            0
 yr_renovated     3842
 zipcode             0
 sqft_living15       0
 sqft_lot15          0
 dtype: int64, (21597, 18))

In [26]:
zips = pd.get_dummies(kc_housing['zipcode'], prefix='zip')

In [28]:
zips.shape

(21597, 70)

In [29]:
kc_housing['yr_built'] = kc_housing['yr_built'].apply(lambda x: 2019 - x)
kc_housing.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,64,0.0,98178,1340,5650
1,2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,68,1991.0,98125,1690,7639
2,2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,86,NaN,98028,2720,8062
3,2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,54,0.0,98136,1360,5000
4,2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,32,0.0,98074,1800,7503


In [39]:
221900 * (.008+.007+.021+.021+.019)

16864.4

In [38]:
(.008+.007+.021+.021+.019)

0.07600000000000001